In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, Reshape, LSTM
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold

In [5]:
data_directory = './data/Train-Test Set 2'

transform = transforms.Compose([
    transforms.Resize((640, 480)),
    transforms.ToTensor(),
])

custom_dataset = ImageFolder(root=data_directory, transform=transform)

In [6]:
torch.manual_seed(42)

In [7]:
batch_size = 32
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [11]:
model2 = Sequential()

model2.add(Conv2D(64, (10, 10), activation='relu', input_shape=(640, 480, 3)))
model2.add(MaxPooling2D((10, 10)))
model2.add(Dropout(.2))
model2.add(Conv2D(128, (10, 10), activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((5, 5)))
model2.add(Dropout(.2))
model2.add(Conv2D(256, (5, 5), activation='relu'))
model2.add(Conv2D(512, (2, 2), activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))
model2.add(Dropout(.1))
model2.add(Reshape((1024, 1)))
model2.add(LSTM(1024))
model2.add(Flatten())
model2.add(Dense(1024, activation='relu'))
model2.add(Dropout(.1))
model2.add(Dense(4, activation='softmax'))

In [12]:
model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 631, 471, 64)      19264     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 63, 47, 64)        0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 63, 47, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 54, 38, 128)       819328    
                                                                 
 batch_normalization_2 (Bat  (None, 54, 38, 128)       512       
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 10, 7, 128)       

In [14]:
train_datagen = ImageDataGenerator(
    # rescale=1./255,
)
test_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    './data/Train-Test Set/Training',
    target_size=(640, 480),
    batch_size=8,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    './data/Train-Test Set/Validation',
    target_size=(640, 480),
    batch_size=8,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    './data/Train-Test Set/Testing',
    target_size=(640, 480),
    batch_size=8,
    class_mode='categorical'
)

Found 169 images belonging to 4 classes.
Found 114 images belonging to 4 classes.
Found 72 images belonging to 4 classes.


In [15]:
model2.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

C:\Users\user\AppData\Local\Temp\ipykernel_10156\1707472198.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model2.fit_generator(


Epoch 1/10
22/22 [==============================] - 346s 16s/step - loss: 1.3925 - accuracy: 0.3373 - val_loss: 2.3865 - val_accuracy: 0.3947
Epoch 2/10
22/22 [==============================] - 345s 16s/step - loss: 1.2964 - accuracy: 0.3964 - val_loss: 1.9934 - val_accuracy: 0.2807
Epoch 3/10
22/22 [==============================] - 356s 16s/step - loss: 2.0097 - accuracy: 0.3136 - val_loss: 1.6162 - val_accuracy: 0.2632
Epoch 4/10
22/22 [==============================] - 360s 16s/step - loss: 2.2637 - accuracy: 0.3550 - val_loss: 1.9255 - val_accuracy: 0.2456
Epoch 5/10
22/22 [==============================] - 367s 17s/step - loss: 2.1337 - accuracy: 0.2840 - val_loss: 1.4881 - val_accuracy: 0.3158
Epoch 6/10
22/22 [==============================] - 369s 17s/step - loss: 1.7686 - accuracy: 0.3018 - val_loss: 1.6306 - val_accuracy: 0.3947
Epoch 7/10
22/22 [==============================] - 372s 17s/step - loss: 1.6728 - accuracy: 0.3314 - val_loss: 1.5712 - val_accuracy: 0.3947
Epoch 

In [16]:
model2.save('cnn_lstm_3947.h5')

C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
from sklearn.metrics import confusion_matrix
import itertools

predictions = model2.predict_generator(test_generator)
y_pred = np.round(predictions).astype(int).flatten()  # Convert probabilities to class labels
# print(y_true, y_pred)
y_true = test_generator.classes
print(y_true)
# cm = confusion_matrix(y_true, y_pred)

# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, cm[i, j],
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')

# plot_confusion_matrix(cm, classes=validation_generator.class_indices.keys())

# plt.show()

C:\Users\user\AppData\Local\Temp\ipykernel_10156\1749941455.py:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model2.predict_generator(test_generator)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3]
